<a href="https://colab.research.google.com/github/hjkim-hufs/hufs-CHN-culture-portfolio/blob/main/%EB%B3%B4%EB%9E%8C%ED%8A%9C%EB%B8%8C_%EC%A0%9C%EB%AA%A9_%EC%96%B8%EC%96%B4%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [1]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 46.1MB/s 
     |████████████████████████████████| 92kB 10.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# konlpy를 실행하기
import konlpy

In [3]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [4]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
## 문서 ID로 실제 파일 불러오기
# 보람튜브 제목 데이터
## https://drive.google.com/file/d/1z6BOjwcFH8uafmDsXkWOESYww3xGVoJp/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1z6BOjwcFH8uafmDsXkWOESYww3xGVoJp'})
rawdata_downloaded.GetContentFile('rawdata1.csv')

In [7]:
# "rawdata1.csv" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('rawdata1.csv',  encoding='CP949') ## sep="\t" 를 삭제함. 입력 데이터는 ", 콤마"를 기준으로 나누고 있음.

In [8]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,no,date,title
0,1.0,2019-07-23,"보람튜브, 95억 강남빌딩 매입…유튜브 광고수입 얼마길래?"
1,2.0,2019-07-23,보람튜브 月 수익 얼마길래?…'장난감 리뷰로 42억 원'
2,3.0,2019-07-23,"보람튜브, ""유아 사생활을 공유한다는 것""…신상털이·명예훼손 우려 도마 위"
3,4.0,2019-07-23,"보람튜브' 유행시킨 '액체괴물' 경보…""일부 발암·독성물질 여전"""
4,5.0,2019-07-23,"보람튜브, 월수익 37억 '명암'…아동학대 논란도"
...,...,...,...
416,NaN,NaN,NaN
417,NaN,NaN,NaN
418,NaN,NaN,NaN
419,NaN,NaN,NaN


In [9]:
원본데이터 = 원본데이터.dropna(axis=0) ## 빈값(NAN) 제거

In [10]:
원본데이터

,no,date,title
0,1.0,2019-07-23,"보람튜브, 95억 강남빌딩 매입…유튜브 광고수입 얼마길래?"
1,2.0,2019-07-23,보람튜브 月 수익 얼마길래?…'장난감 리뷰로 42억 원'
2,3.0,2019-07-23,"보람튜브, ""유아 사생활을 공유한다는 것""…신상털이·명예훼손 우려 도마 위"
3,4.0,2019-07-23,"보람튜브' 유행시킨 '액체괴물' 경보…""일부 발암·독성물질 여전"""
4,5.0,2019-07-23,"보람튜브, 월수익 37억 '명암'…아동학대 논란도"
...,...,...,...
396,397.0,2019-09-11,"“‘보람튜브’는 기적, 현실은 냉혹” 유튜버가 말하는 유튜브 빛과 그림자"
397,398.0,2019-09-23,6살 보람이는 강남 95억 빌딩 건물주!
398,399.0,2019-09-30,월 37억 벌던 '보람튜브' 광고수입 확 준다
399,400.0,2019-09-30,보람튜브'에 난리가 날 것 같다… 유튜브가 충격적인 정책을 발표했다


# 형태소 분석

In [14]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [17]:
# "원본데이터"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터.iterrows(): # 원본데이터의 컬럼(열)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    # 바로: 대상 텍스트가 있는 열은 title or text로 각각 3번째 or 4번째 열입니다.
    분류 = row[1] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    # 바로 : 분류 기준을 신문사로 할 경우 2, 날짜로 할 경우 1 입니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [13]:
# "형태소" 변수의 저장 내용 확인
형태소

NameError: ignored

In [18]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,"""",Punctuation,2019-09-30,1
1,제,Noun,2019-09-30,1
2,2,Number,2019-09-30,1
3,의,Noun,2019-09-30,1
4,보람,Noun,2019-09-30,1
5,좌,Noun,2019-09-30,1
6,없다,Adjective,2019-09-30,1
7,"?!""···",Punctuation,2019-09-30,1
8,유튜브,Noun,2019-09-30,1
9,",",Punctuation,2019-09-30,1


In [19]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[      0            1        type  count
 0    보람         Noun  2019-07-23      1
 1    튜브         Noun  2019-07-23      1
 2     ,  Punctuation  2019-07-23      1
 3   95억       Number  2019-07-23      1
 4    강남         Noun  2019-07-23      1
 5    빌딩         Noun  2019-07-23      1
 6    매입         Noun  2019-07-23      1
 7     …  Punctuation  2019-07-23      1
 8   유튜브         Noun  2019-07-23      1
 9    광고         Noun  2019-07-23      1
 10   수입         Noun  2019-07-23      1
 11   얼마         Noun  2019-07-23      1
 12    길         Noun  2019-07-23      1
 13    래         Josa  2019-07-23      1
 14    ?  Punctuation  2019-07-23      1,
       0            1        type  count
 0    보람         Noun  2019-07-23      1
 1    튜브         Noun  2019-07-23      1
 2     月      Foreign  2019-07-23      1
 3    수익         Noun  2019-07-23      1
 4    얼마         Noun  2019-07-23      1
 5     길         Noun  2019-07-23      1
 6     래         Josa  2019-07-23      1
 7   ?…'  Punct

In [20]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [21]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,보람,Noun,2019-07-23,1
1,튜브,Noun,2019-07-23,1
2,",",Punctuation,2019-07-23,1
3,95억,Number,2019-07-23,1
4,강남,Noun,2019-07-23,1
...,...,...,...,...
16,',Punctuation,2019-09-30,1
17,수익,Noun,2019-09-30,1
18,',Punctuation,2019-09-30,1
19,반토막,Noun,2019-09-30,1


In [22]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [23]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,보람,Noun,2019-07-23,1
1,튜브,Noun,2019-07-23,1
2,",",Punctuation,2019-07-23,1
3,95억,Number,2019-07-23,1
4,강남,Noun,2019-07-23,1
...,...,...,...,...
16,',Punctuation,2019-09-30,1
17,수익,Noun,2019-09-30,1
18,',Punctuation,2019-09-30,1
19,반토막,Noun,2019-09-30,1


In [24]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [25]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소   품사           분류        
\n    Foreign      2019-07-26    1
                   2019-08-07    1
                   2019-08-25    1
!     Punctuation  2019-07-25    2
                   2019-09-23    1
                                ..
훔치는   Verb         2019-07-24    2
휘둥그레  Noun         2019-07-23    2
힘     Noun         2019-07-23    3
                   2019-07-24    1
＋     Foreign      2019-07-23    1
Name: 카운트, Length: 2384, dtype: int64

In [26]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [27]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소  품사          분류             
\n   Foreign     2019-07-26    1
                 2019-08-07    1
                 2019-08-25    1
!    Punctuation 2019-07-25    2
                 2019-09-23    1
...                          ...
훔치는  Verb        2019-07-24    2
휘둥그레 Noun        2019-07-23    2
힘    Noun        2019-07-23    3
                 2019-07-24    1
＋    Foreign     2019-07-23    1

[2384 rows x 1 columns]

In [28]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')